# Aspect-Based Sentiment Analysis with DistilBERT

Aspect-based sentiment analysis (ABSA) is a text analysis technique that categorizes data by aspect and identifies the sentiment attributed to each one
- Sentiments: positive or negative or neutral opinions about a particular aspect
- Aspects: the category, feature, or topic that is being talked about

BERT (Bidirectional Embedding Representations from Transformers) is a new model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. 

DistilBERT is a transformers model, smaller and faster than BERT, which was pretrained on the same corpus in a self-supervised fashion, using the BERT base model as a teacher. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts using the BERT base model.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

from tensorflow.keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model


2022-07-17 15:26:15.298323: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-07-17 15:26:15.298435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Install the huggingface transformer package.

In [2]:
!pip install transformers

## Preprocessing and Tokenization

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for that model. 
We can get the DistilBERT tokenizer from **DistilBertTokenizer.from_pretrained** function.

In [3]:
from transformers import DistilBertTokenizer 
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids) # specific ids of each token in the sentence
print(masks) # attention mask- which token to attend to. 1-attend to, 0-don't attend, padding
print(segments) # sentence segments-[sent1,sent2]-> [0,0,0,0,1,1,1,1]
print(tokenizer.convert_ids_to_tokens(ids),"\n") # [CLS] tag is used by BERT for classification tasks

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Downloading and preprocessing data

The dataset used is the aspect-term sentiment analysis (ATSA) dataset, which consists of 12522 labeled reviews. These are split into 11186 reviews for training and 1336 reviews for testing and validation, respectively. 

For ATSA, the annotators extract aspect terms in the sentences and label the sentiment polarities with respect to the  aspect  terms.   The  sentences  that  consist  of only one aspect term or multiple aspects with the same  sentiment  polarities  are  deleted.  ATSA also provides the start and end positions in a sentence for each aspect term.

In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/train.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/val.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/test.xml')


# The code is modified from https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data_process/utils.py
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = [text , term,  polarity , start , end]
            data.append(piece)
    return data
train = parse_sentence_term("train.xml",True)
dev = parse_sentence_term("val.xml",True)
test = parse_sentence_term("test.xml",True)

print("Training entries: {}".format(len(train)))
print("Test entries: {}".format(len(test)))


Training entries: 11186
Test entries: 1336


We now can start playing around with the data, let’s first see some examples:

In [6]:
print("SENTENCE \t ASPECT \t LABLE \t ASPECT-START-INDEX \t ASPECT-END-INDEX")
print(train[0])
print(train[1])
print(train[2])
print(train[3])
print(train[4])

SENTENCE 	 ASPECT 	 LABLE 	 ASPECT-START-INDEX 	 ASPECT-END-INDEX
['the decor is not special at all but their food and amazing prices make up for it.', 'decor', 'negative', '4', '9']
['the decor is not special at all but their food and amazing prices make up for it.', 'food', 'positive', '42', '46']
['the decor is not special at all but their food and amazing prices make up for it.', 'prices', 'positive', '59', '65']
['when tables opened up, the manager sat another party before us.', 'tables', 'neutral', '5', '11']
['when tables opened up, the manager sat another party before us.', 'manager', 'negative', '27', '34']


According to the BERT tokenize function above, we can convert the tweet text and topic words to integers:


In [7]:
train_sentences, train_aspect = [i[0] for i in train], [i[1] for i in train]
dev_sentences, dev_aspect = [i[0] for i in dev], [i[1] for i in dev]
test_sentences, test_aspect = [i[0] for i in test], [i[1] for i in test]

x_train_review_int, x_train_review_masks, _ = tokenize(train_sentences, tokenizer)
x_train_aspect_int, x_train_aspect_masks, _ = tokenize(train_aspect, tokenizer)

x_dev_review_int, x_dev_review_masks, _ = tokenize(dev_sentences, tokenizer)
x_dev_aspect_int, x_dev_aspect_masks, _ = tokenize(dev_aspect, tokenizer)

x_test_review_int, x_test_review_masks, _ = tokenize(test_sentences, tokenizer)
x_test_aspect_int, x_test_aspect_masks, _ = tokenize(test_aspect, tokenizer)

assert len(x_train_aspect_int) == len(train)
assert len(x_train_aspect_masks) == len(x_train_aspect_int)
assert len(x_test_aspect_int) == len(test)
assert len(x_test_aspect_masks) == len(x_test_aspect_int)
print("x_dev_aspect_int[0]:")
print(x_dev_aspect_int[0])
print("x_dev_aspect_masks[0]:")
print(x_dev_aspect_masks[0])
print("x_dev_review_int[0]:")
print(x_dev_review_int[0])
print("x_dev_review_masks[0]:")
print(x_dev_review_masks[0])

x_dev_aspect_int[0]:
[ 101 8974  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_aspect_masks[0]:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_review_int[0]:
[  101  2044  1037  323

One-hot encode the labels, using 4 (Binary:100) to represent "positive", 2 (Binary:010) for "neutral", and 1 (Binary:001) for "negative". Then we can convert the labels to numbers.

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append([0,0,1])
    elif example[2].lower() == "neutral":
      y.append([0,1,0])
    else:
      # assert example[2].lower() == "positive"
      y.append([1,0,0])
  return y
  
y_train = label2int(train)
y_dev = label2int(dev)
y_test = label2int(test)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


There are two x (review and aspect) to input the model. Combine the review and aspect into one sentence and then input it into the model.

In [9]:
tokenize([i[0]+' [SEP] '+i[1] for i in dev][0], tokenizer)
# tokenize('<SEP>', tokenizer)

(array([  101,  2044,  1037,  3232,  1997,  8974,  1010,  1996, 18726,
         1011,  1011,  1045,  2066,  1996, 27940,  1013, 24792,  2621,
         4897,  1998,  1996, 13675, 11514,  6508, 26852,  1011,  1011,
         2175,  2091,  2307,  1012,   102,  8974,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [10]:
train_sentences = [i[0]+' [SEP] '+i[1] for i in train]
print(train_sentences[0])
dev_sentences = [i[0]+' [SEP] '+i[1] for i in dev]
print(dev_sentences[0])
test_sentences = [i[0]+' [SEP] '+i[1] for i in test]
print(test_sentences[0])

x_train_int, x_train_masks, _ = tokenize(train_sentences, tokenizer)

x_dev_int, x_dev_masks, _ = tokenize(dev_sentences, tokenizer)

x_test_int, x_test_masks, _ = tokenize(test_sentences, tokenizer)

x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0]) # senetnce + aspect

the decor is not special at all but their food and amazing prices make up for it. [SEP] decor
after a couple of drinks, the apps--i like the lobster/mango summer roll and the cripsy squid--go down great. [SEP] drinks
the food was served promptly but the meal wasn't rushed - we had plenty of time to enjoy the appetizers and our entrees as well as sit and chat while finishing up our drinks even after we paid. [SEP] food
[  101  2044  1037  3232  1997  8974  1010  1996 18726  1011  1011  1045
  2066  1996 27940  1013 24792  2621  4897  1998  1996 13675 11514  6508
 26852  1011  1011  2175  2091  2307  1012   102  8974   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0 


## Model 1: Neural bag of words using BERT

In [11]:
# custom averagepooling layer with masking
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [12]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
    distil_bert = 'distilbert-base-uncased'
    config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
    config.output_hidden_states = False
    return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [13]:
import tensorflow as tf
hdepth=16  # got 16 by trial and test
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
    input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

    bert_embeddings = get_BERT_layer()
    embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

    pooled_sent=GlobalAveragePooling1DMasked()(embedded_sent)
    hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(pooled_sent) # Sigmoid
    label=Dense(3,input_shape=(hdepth,),activation='softmax',kernel_initializer='glorot_uniform')(hidden_output)
    return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model1_BERT')

use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model1 = create_bag_of_words_BERT()
        optimizer2 = tf.keras.optimizers.Adam(lr=5e-5)
        model1.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
    model1 = create_bag_of_words_BERT()
    model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.summary() 




2022-07-17 15:27:09.657387: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-17 15:27:09.660867: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-07-17 15:27:09.660909: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-17 15:27:09.660941: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (578c28433959): /proc/driver/nvidia/version does not exist
2022-07-17 15:27:09.664312: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

2022-07-17 15:27:36.860754: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the 

Model: "Model1_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_masked (None, 768)          0           tf_distil_bert_model[0]

In [14]:

history = model1.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 85s 2s/step - loss: 0.6647 - accuracy: 0.3820 - val_loss: 0.5487 - val_accuracy: 0.5946
Epoch 2/30
22/22 [==============================] - 3s 132ms/step - loss: 0.5399 - accuracy: 0.5973 - val_loss: 0.4960 - val_accuracy: 0.6577
Epoch 3/30
22/22 [==============================] - 3s 132ms/step - loss: 0.4811 - accuracy: 0.6912 - val_loss: 0.4479 - val_accuracy: 0.7508
Epoch 4/30
22/22 [==============================] - 3s 134ms/step - loss: 0.4285 - accuracy: 0.7722 - val_loss: 0.4256 - val_accuracy: 0.7860
Epoch 5/30
22/22 [==============================] - 3s 132ms/step - loss: 0.3961 - accuracy: 0.8199 - val_loss: 0.4157 - val_accuracy: 0.8101
Epoch 6/30
22/22 [==============================] - 3s 130ms/step - loss: 0.3788 - accuracy: 0.8519 - val_loss: 0.4119 - val_accuracy: 0.8071
Epoch 7/30
22/22 [==============================] - 3s 133ms/step - loss: 0.3580 - accuracy: 0.8786 - val_loss: 0.4103 - val_accuracy: 0.8108
Epoch 8/

In [15]:
results = model1.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 6s 70ms/step - loss: 0.3941 - accuracy: 0.8189
[0.3941439092159271, 0.8188623189926147]


## Model 2: LSTM with BERT

In [16]:
import tensorflow as tf
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
    input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

    bert_embeddings = get_BERT_layer()
    embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

    # pooled_sent=GlobalAveragePooling1DMasked()(embedded_sent)
    lstm_layer = LSTM(100)(embedded_sent)
    # hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(lstm_layer) # Sigmoid
    label=Dense(3,input_shape=(hdepth,),activation='softmax',kernel_initializer='glorot_uniform')(lstm_layer)
    return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model2 = create_bag_of_words_BERT()
        optimizer2 = tf.keras.optimizers.Adam(lr=5e-5)
        model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
    model2 = create_bag_of_words_BERT()

    model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model2.summary() 


2022-07-17 15:30:48.029693: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2022-07-17 15:30:48.030089: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}
2022-07-17 15:30:48.034873: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2022-07-17 15:30:48.034920: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a 

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 100)          347600      tf_distil_bert_model_1[

In [17]:
history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 87s 2s/step - loss: 0.6245 - accuracy: 0.4616 - val_loss: 0.4935 - val_accuracy: 0.6314
Epoch 2/30
22/22 [==============================] - 3s 137ms/step - loss: 0.4675 - accuracy: 0.6462 - val_loss: 0.3737 - val_accuracy: 0.7462
Epoch 3/30
22/22 [==============================] - 3s 136ms/step - loss: 0.3371 - accuracy: 0.7753 - val_loss: 0.3169 - val_accuracy: 0.7943
Epoch 4/30
22/22 [==============================] - 3s 137ms/step - loss: 0.2808 - accuracy: 0.8249 - val_loss: 0.3074 - val_accuracy: 0.8018
Epoch 5/30
22/22 [==============================] - 3s 136ms/step - loss: 0.2323 - accuracy: 0.8589 - val_loss: 0.3071 - val_accuracy: 0.8191
Epoch 6/30
22/22 [==============================] - 3s 136ms/step - loss: 0.1881 - accuracy: 0.8931 - val_loss: 0.3146 - val_accuracy: 0.8168
Epoch 7/30
22/22 [==============================] - 3s 136ms/step - loss: 0.1495 - accuracy: 0.9190 - val_loss: 0.3257 - val_accuracy: 0.8258
Epoch 8/

In [18]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 6s 76ms/step - loss: 0.4959 - accuracy: 0.8256
[0.49594008922576904, 0.8255988359451294]


**LSTM accuracy(Model 2): 82.78% , Dense accuracy(Model 1): 82.63%**
<br>Adding LSTM layer doesn't seem to improve the accuracy. It may be due to the fact that BERT embedding are already good as its trained on huge corpus. And adding LSTM layers wont affect the accuracy much. There is a good chance for the training to increase if we train the bert layers based on our data. But its not feasible as it will take huge amount of time due to millions of BERT trainable parameters.